In [6]:
import torch
from model import *
import numpy as np
from utils import *
from rans import *
from tqdm import tqdm
torch.use_deterministic_algorithms(True)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
opt = {}


opt["device"] = torch.device("cpu")
opt["if_cuda"] = False


In [3]:
opt['data_set']='BinaryMNIST'
opt['dataset_path']='../data/'
opt['save_path']='save/'
opt['z_dim']=10
opt['if_bn']=False
opt['h_layer_num']=2


load_name='BinaryMNIST_epoch30.pth'
model=VAE(opt).to(opt['device'])
model.load_state_dict(torch.load(opt['save_path']+load_name))


test_data=torchvision.datasets.MNIST(opt['dataset_path'], train=False, download=False)

c_opt={}
c_opt['data_set']=opt['data_set']
c_opt['obs_dis']='Bernoulli'
c_opt['device']=opt["device"]
c_opt['s_prec']=64
c_opt['t_prec']=32
c_opt['p_prec']=10
c_opt['log_num_discretization']=10
c_opt['z_dim']=10
c_opt['x_dim']=784
c_opt['discretization_scheme']=std_gaussian_buckets(c_opt['log_num_discretization'])
c_opt['discretization_centres']=std_gaussian_centres(c_opt['log_num_discretization'])
c_opt['channel_auto']=False

for i in range(0,1):
    test_img=torch.round(test_data.test_data[i:i+1].view(1,784)/255)
    with torch.no_grad():
        print('Theo BPD',-model(test_img.view(1,1,28,28)).item()/784)


    ansstack = ANSStack(c_opt['s_prec'], c_opt['t_prec'],c_opt['p_prec'],randominit=True)

    stack_init_size=ansstack.get_length()
    ansstack =BBCompression(test_img.numpy(),ansstack,model,c_opt)

    print('True BPD',(ansstack.get_length()-stack_init_size)/784)

    decode_img,ansstack=BBDecompression(ansstack,model,c_opt)

    print('Error',(np.int64(decode_img).reshape(784)-np.int64(test_img).reshape(784)).sum().item())



Theo BPD 0.1033024787902832
True BPD 0.10714285714285714
Error 0


In [5]:
opt['data_set']='MNIST'
opt['dataset_path']='../data/'
opt['save_path']='./save/'
opt['z_dim']=10
opt['if_bn']=False
opt['h_layer_num']=2


load_name='MNIST_epoch30.pth'
model=VAE(opt).to(opt['device'])
model.load_state_dict(torch.load(opt['save_path']+load_name))



test_data=torchvision.datasets.MNIST(opt['dataset_path'], train=False, download=False)


c_opt={}
c_opt['data_set']=opt['data_set']
c_opt['obs_dis']='Logistic'
c_opt['device']=opt["device"]
c_opt['s_prec']=64
c_opt['t_prec']=32
c_opt['p_prec']=14
c_opt['log_num_discretization']=10
c_opt['z_dim']=10
c_opt['x_dim']=784
c_opt['discretization_scheme']=std_gaussian_buckets(c_opt['log_num_discretization'])
c_opt['discretization_centres']=std_gaussian_centres(c_opt['log_num_discretization'])
c_opt['obs_dis']='Logistic'
c_opt['channel_auto']=False


for i in range(0,1):
    print(i)
    test_img=test_data.test_data[i:i+1].view(1,784)/255.
    with torch.no_grad():
        print('Theo BPD',-model(test_img.view(1,1,28,28)).item()/784)

    ansstack = ANSStack(c_opt['s_prec'], c_opt['t_prec'],c_opt['p_prec'],randominit=True)

    stack_init_size=ansstack.get_length()
    ansstack =BBCompression(test_img.numpy(),ansstack,model,c_opt)

    print('True BPD',(ansstack.get_length()-stack_init_size)/784)

    decode_img,ansstack=BBDecompression(ansstack,model,c_opt)

    origin_img=np.int64(test_img.view(784)*255)
    recover_img=np.int64(torch.tensor(decode_img*255).view(784))
    print('Error',(origin_img-recover_img).sum())



0
Theo BPD 1.2341353747309471
True BPD 1.3380102040816326
Error 0


In [ ]:
opt = {}

opt["device"] = torch.device("cpu")
opt["if_cuda"] = False
opt['data_set']='CIFAR'
opt['dataset_path']='../data/cifar10'
opt['save_path']='./save/'
opt['result_path']='./results/'
opt['epochs'] = 1
opt['batch_size'] = 100
opt['test_batch_size']=1000
opt['if_regularizer']=False
opt['load_model']=False
opt['lr']=1e-3
opt['z_dim']=256
opt['net']='res'
opt['SelfConsistPhase']=False
opt['x_dis']='logistic_cato'
opt['if_save']=True



load_name='CIFARlogistic_cato_epoch10.pth'
model=VAE(opt).to(opt['device'])
model.load_state_dict(torch.load(opt['save_path']+load_name))


test_data=torchvision.datasets.CIFAR10(opt['dataset_path'], train=False, download=False)


c_opt={}
c_opt['data_set']=opt['data_set']
c_opt['device']=opt["device"]
c_opt['s_prec']=64
c_opt['t_prec']=32
c_opt['p_prec']=16
c_opt['log_num_discretization']=16
c_opt['z_dim']=256
c_opt['x_dim']=3072
c_opt['discretization_scheme']=std_gaussian_buckets(c_opt['log_num_discretization'])
c_opt['discretization_centres']=std_gaussian_centres(c_opt['log_num_discretization'])
c_opt['obs_dis']='Logistic_cato'

model.eval()
BPD_list=[]
for i in range(0,100):
    print(i)
    test_img=torch.tensor(test_data.test_data[0:1],dtype=torch.float32).permute(0,3,1,2).contiguous().view(1,c_opt['x_dim'])/255.

    # print('origin',np.uint64(test_img.view(-1)[0:215].numpy()*255).sum())
    with torch.no_grad():
        model.eval()
        print(test_img.view(1,3,32,32).size())
        print('Theo BPD',-model(test_img.view(1,3,32,32)).item()/c_opt['x_dim'])

    ansstack = ANSStack(c_opt['s_prec'], c_opt['t_prec'],c_opt['p_prec'],randominit=True)

    stack_init_size=ansstack.get_length()
    ansstack =BBCompression(test_img.numpy(),ansstack,model,c_opt)

    print('True BPD',(ansstack.get_length()-stack_init_size)/c_opt['x_dim'])

    decode_img,ansstack=BBDecompression(ansstack,model,c_opt)

    origin_img=np.int64(test_img.view(c_opt['x_dim'])*255)
    recover_img=np.int64(torch.tensor(decode_img*255).view(c_opt['x_dim']))
    # plt.imshow(recover_img.reshape(28,28))
    # plt.show()
    print('Error',(origin_img-recover_img).sum())

    BPD_list.append((ansstack.get_length()-stack_init_size)/c_opt['x_dim'])
    
print(np.mean(BPD_list))